### 4.  VGG FLOPs

In [1]:
import torch

torch.cuda.is_available()

True

In [2]:
import math
import torch
import torch.nn as nn
from torch.autograd import Variable

default_cfg = {
    11: [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512],
    13: [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512],
    16: [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512],
    19: [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512],
}


def make_layers(cfg):
    layers = []
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1, bias=False)
            layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)


class vgg(nn.Module):
    def __init__(self, dataset='cifar10', depth=19, cfg=None, conv_cfg=None):
        """
        note: default initialize weight & have batch normalization

        :param dataset: `cifar10` or `cifar100`
        :param depth: `11`, `13`, '16', or `19` (default)
        :param cfg: vgg model convolutional layer's channel config
        :param conv_cfg:
            like [2, 4, 8, 12]
            return convolutional layer's channel config (index starts at 1)

        """
        super(vgg, self).__init__()

        # model value
        self.conv_cfg = conv_cfg

        # model config
        if cfg is None:
            cfg = default_cfg[depth]

        # model feature
        self.feature = make_layers(cfg)

        # model classifier
        if dataset == 'cifar10':
            num_classes = 10
        elif dataset == 'cifar100':
            num_classes = 100
        else:
            raise ValueError('Model `dataset` parameter is Error!')
        self.classifier = nn.Linear(cfg[-1], num_classes)

        # model initialize weight
        self._initialize_weights()

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(0.5)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()

    def forward(self, x):
        # model feature
        conv_value = []
        if self.conv_cfg:
            conv_idx = 0
            for k, m in enumerate(self.feature):
                x = m(x)
                if isinstance(m, nn.Conv2d):
                    conv_idx += 1
                    if conv_idx in self.conv_cfg:
                        conv_value.append(x.clone())
        else:
            x = self.feature(x)

        # model classifier
        x = nn.AvgPool2d(2)(x)
        x = x.view(x.size(0), -1)
        y = self.classifier(x)

        # return value
        if len(conv_value):
            return y, conv_value
        return y


# if __name__ == '__main__':
#     net = vgg(conv_cfg=[2, 4, 8, 12])
#     input = Variable(torch.FloatTensor(64, 3, 32, 32))
#     output, value = net(input)
#     print('y.data.shape: {}, value length: {}'.format(input.data.shape, len(value)))



In [3]:
import os

def resume_model(resume_file):
    if not os.path.isfile(resume_file):
        raise ValueError("Resume model file is not found at '{}'".format(resume_file))
    print("=> loading checkpoint '{}'".format(resume_file))
    checkpoint = torch.load(resume_file)
    if 'epoch' in checkpoint:
        start_epoch = checkpoint['epoch']
    else:
        start_epoch = None
        
    if 'best_prec1' in checkpoint:
        best_prec1 = checkpoint['best_prec1']
    else:
        best_prec1 = None
        
    if 'state_dict' in checkpoint:
        state_dict = checkpoint['state_dict']
    else:
        state_dict = None
    
    if 'optimizer' in checkpoint:
        opti_dict = checkpoint['optimizer']
    else:
        opti_dict = None
        
    if 'cfg' in checkpoint:
        cfg = checkpoint['cfg']
        print("-> model cfg is loading...\n cfg: {}".format(list(cfg)))
    else:
        cfg = None
        print("-> not found model cfg...")
    print("=>  epoch {} Prec1: {}".format(start_epoch, best_prec1))
    return state_dict, opti_dict, start_epoch, best_prec1, cfg


data_path = {
    'root': r'D:\Project\Gitee\network-slimming\logs',
    'bn': [
        'bn_prune_vgg19_cifar10_percent_0.7',
        'bn_prune_vgg19_cifar100_percent_0.5',
    ],
    'at': [
        'at_prune_vgg19_cifar10_percent_0.7',
        'at_prune_vgg19_cifar100_percent_0.5',
    ],
    'flie': 'model_best.pt',
    'file2': 'model_best.pth.tar',
    'file3': 'pruned.pt',
}


file_path = os.path.join(data_path['root'], data_path['at'][1], data_path['file3'])
state_dict, opti_dict, start_epoch, best_prec1, cfg = resume_model(file_path)

=> loading checkpoint 'D:\Project\Gitee\network-slimming\logs\at_prune_vgg19_cifar100_percent_0.5\pruned.pt'
-> model cfg is loading...
 cfg: [21, 59, 'M', 107, 125, 'M', 241, 229, 203, 205, 'M', 309, 239, 142, 171, 'M', 153, 143, 166, 238]
=>  epoch None Prec1: None


In [6]:
model = vgg(depth=19, dataset='cifar100', cfg=cfg, conv_cfg=None)
model.load_state_dict(state_dict)
model.eval()

vgg(
  (feature): Sequential(
    (0): Conv2d(3, 21, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(21, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(21, 59, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(59, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(59, 107, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (8): BatchNorm2d(107, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(107, 125, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (11): BatchNorm2d(125, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilatio

In [7]:
from torchstat import stat

stat(model, (3, 32, 32))

      module name  input shape output shape     params memory(MB)           MAdd          Flops  MemRead(B)  MemWrite(B) duration[%]   MemR+W(B)
0       feature.0    3  32  32   21  32  32      567.0       0.08    1,139,712.0      580,608.0     14556.0      86016.0       0.00%    100572.0
1       feature.1   21  32  32   21  32  32       42.0       0.08       86,016.0       43,008.0     86184.0      86016.0       0.00%    172200.0
2       feature.2   21  32  32   21  32  32        0.0       0.08       21,504.0       21,504.0     86016.0      86016.0       0.00%    172032.0
3       feature.3   21  32  32   59  32  32    11151.0       0.23   22,776,832.0   11,418,624.0    130620.0     241664.0       0.00%    372284.0
4       feature.4   59  32  32   59  32  32      118.0       0.23      241,664.0      120,832.0    242136.0     241664.0       0.00%    483800.0
5       feature.5   59  32  32   59  32  32        0.0       0.23       60,416.0       60,416.0    241664.0     241664.0       0.0

In [8]:
import torch
from thop import profile
from thop import clever_format


input = torch.randn(1, 3, 32, 32)
flops, params = profile(model, inputs=(input, ))
print(flops, params)
flops, params = clever_format([flops, params], "%.2f")
flops, params

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[WARN] Cannot find rule for <class '__main__.vgg'>. Treat it as zero Macs and zero Params.
189846232.0 4539392.0


('189.85M', '4.54M')